# **Fast Influence-based Coarsening for Large Networks**

---

**Definition 4.2 (Merging node pairs).** Let $Nb^i(v)$ (respectively $Nb^o(v)$) denote the set of in-neighbors (resp. out-neighbors) of a vertex $v$. Let $v^i_u = w(u,v)$ and $v^o_u = w(v,u)$ denote the weight of the corresponding edges. If the node pair $(a,b)$ is now contracted to a new vertex $c$, and0 $w(a,b) = \beta_1$ and $w(b,a) = \beta_2$, then the new edges are weighted as:

$c^i_t = \begin{cases} \frac{(1+\beta_1)a^i_t}{2} & \forall t \in Nb^i(a)\setminus Nb^i(b) \\ \frac{(1+\beta_2)b^i_t}{2} & \forall t \in Nb^i(b)\setminus Nb^i(a) \\ \frac{(1+\beta_1)a^i_t + (1+\beta_2)b^i_t}{4} & \forall t \in Nb^i(a) \cap Nb^i(b)\end{cases}$

$c^o_t = \begin{cases} \frac{(1+\beta_2)a^o_t}{2} & \forall t \in Nb^o(a)\setminus Nb^o(b) \\ \frac{(1+\beta_1)b^o_t}{2} & \forall t \in Nb^o(b)\setminus Nb^o(a) \\ \frac{(1+\beta_2)a^o_t + (1+\beta_1)b^o_t}{4} & \forall t \in Nb^o(a) \cap Nb^o(b)\end{cases}$

---

**Proposition 5.2 (Score estimate).**  The score of a node pair $score(a,b)$ can be approximated as

$$
\Delta \lambda_{(a,b)} =
\frac{
    -\lambda (u_a v_a + u_b v_b) + v_a \vec{u}^T \vec{c}^o + \beta_2 u_a v_b + \beta_1 u_b v_a
}{
    \vec{v}^T \vec{u} - (u_a v_a + u_b v_b)
}
$$

*(ignoring second order terms)*.

---

**Proposition 5.3.**  Using the re-weighting scheme as defined in Definition 4.2, if $c$ denotes de new vertex created by merging nodes $\{a,b\}$ and $\vec{c}^o$ the out-adjacency vector of $c$,

$$
\vec{u}^T \vec{c}^o =
\frac{(1 + \beta_2)}{2} \left( \lambda u_a - \beta_1 u_b \right)
+ \frac{(1 + \beta_1)}{2} \left( \lambda u_b - \beta_2 u_a \right)
$$

where $\beta_1$ is the weight of edge $(a,b)$ and $\beta_2$ is the weight of the edge $(b,a)$.

---

$
\begin{array}{l}
\textbf{Algorithm:} \ \text{Coarsening Algorithm - CoarseNet}(G, \alpha) \\
\textbf{Input:} \ \text{A directed, weighted graph } G = (V, E, w), \ \text{a reduction factor } \alpha \\
\textbf{Output:} \ \text{Coarsened graph } G^{\alpha}_{\text{coarse}} = (V', E', w') \\
1:\quad i \gets 0 \\
2:\quad n \gets |V| \\
3:\quad G' \gets G \\
4:\quad \textbf{for each} \ \text{adjacent pair of nodes } a, b \in V \ \textbf{do} \\
5:\qquad \text{Compute } score(a, b) \ \text{using Section 5.1} \\
6:\quad \pi \gets \text{ordering of node pairs in increasing order of } score \\
7:\quad \textbf{while} \ i \leq \alpha n \ \textbf{do} \\
8:\qquad (a, b) \gets \pi(i) \\
9:\qquad G' \gets \text{Contract}_{G'}(a, b) \\
10:\qquad i \gets i + 1 \\
11:\quad \textbf{return} \ G^{\alpha}_{\text{coarse}} = G'
\end{array}
$

---

In [1]:
import networkx as nx
import numpy as np
from scipy.sparse.linalg import eigs
import warnings

In [2]:
def _calculate_eigensystem(G, nodelist):
    """
    Calcula lambda y los autovectores.
    Cambia de método para grafos muy pequeños para evitar errores.
    """
    num_nodes = G.number_of_nodes()

    if 0 < num_nodes <= 2:
        try:
            A_dense = nx.to_numpy_array(G, nodelist=nodelist, weight='weight')
            eigenvalues = np.linalg.eigvals(A_dense)
            lambda_val = np.max(np.real(eigenvalues))
            return lambda_val, np.array([]), np.array([])
        except Exception as e:
            warnings.warn(f"Cálculo con Numpy falló: {e}")
            return None, None, None

    A = nx.to_scipy_sparse_array(G, nodelist=nodelist, weight='weight', dtype=np.float64)
    try:
        lambda_val, u = eigs(A, k=1, which='LR')
        _, v = eigs(A.T, k=1, which='LR')
    except Exception as e:
        warnings.warn(f"Cálculo de valores propios falló: {e}.")
        return None, None, None

    lambda_val = np.real(lambda_val[0])
    u = np.real(u[:, 0])
    v = np.real(v[:, 0])
    return lambda_val, u, v

In [ ]:
def _contract_pair(G: nx.DiGraph, a, b):
    """
    Contract nodes (a, b) into a super-node (a,b), updating edges as in Def. 4.2.
    """
    new_node = (a, b)
    G_new = G.copy()

    def get_w(u, v):
        d = G_new.get_edge_data(u, v)
        return d.get("weight", 0.0) if d else 0.0

    beta1 = get_w(a, b)  # weight a->b
    beta2 = get_w(b, a)  # weight b->a

    def merge_side(neigh_a, neigh_b, w_a, w_b, add_edge, factor_a, factor_b):
        """
        Combine edges to or from a,b toward a target t.
        neigh_a, neigh_b: neighbor sets of a and b for the chosen direction.
        w_a(t), w_b(t): weights along that direction (a<->t or t<->a).
        add_edge(t, w): how to add the new edge for target t.
        factor_a, factor_b: multiplicative factors for weights from a and b.
        """
        for t in (neigh_a | neigh_b):
            if t in (a, b):
                continue
            ina = t in neigh_a
            inb = t in neigh_b

            if ina and not inb:
                new_w = factor_a * w_a(t) / 2.0
            elif inb and not ina:
                new_w = factor_b * w_b(t) / 2.0
            else:  # both present
                new_w = (factor_a * w_a(t) + factor_b * w_b(t)) / 4.0

            if new_w != 0:
                add_edge(t, round(new_w, 4))

    # Outgoing edges: (a,b) -> t
    succ_a = set(G_new.successors(a))
    succ_b = set(G_new.successors(b))
    merge_side(
        neigh_a = succ_a,
        neigh_b = succ_b,
        w_a = lambda t: get_w(a, t),
        w_b = lambda t: get_w(b, t),
        add_edge = lambda t, w: G_new.add_edge(new_node, t, weight=w),
        factor_a = 1 + beta2,   # matches original use with weight a->t
        factor_b = 1 + beta1,   # matches original use with weight b->t
    )

    # Incoming edges: t -> (a,b)
    pred_a = set(G_new.predecessors(a))
    pred_b = set(G_new.predecessors(b))
    merge_side(
        neigh_a = pred_a,
        neigh_b = pred_b,
        w_a = lambda t: get_w(t, a),
        w_b = lambda t: get_w(t, b),
        add_edge = lambda t, w: G_new.add_edge(t, new_node, weight=w),
        factor_a = 1 + beta1,   # matches original use with weight t->a
        factor_b = 1 + beta2,   # matches original use with weight t->b
    )

    G_new.remove_node(a)
    G_new.remove_node(b)
    return G_new

In [4]:
def coarsenet(G: nx.DiGraph, alpha: float):
    if not (0 < alpha < 1):
        raise ValueError("El factor de reducción alpha debe estar entre 0 y 1.")
    if G.number_of_nodes() < 2:
        return G.copy()

    G_coarse = G.copy()

    nodelist = list(G_coarse.nodes())
    node_to_idx = {node: i for i, node in enumerate(nodelist)}
    lambda_val, u, v = _calculate_eigensystem(G_coarse, nodelist)
    print(f"Autovalor inicial (Lambda): {lambda_val:.4f}")

    if lambda_val is None:
        return G_coarse

    v_dot_u = v.T @ u
    if abs(v_dot_u) < 1e-15:
        warnings.warn("El producto v^T*u es cercano a cero.")

    scores = []
    # Usamos un set para evitar calcular el score del mismo par no ordenado dos veces (e.g., (a,b) y (b,a))
    scored_pairs_set = set()
    for a_node, b_node in G_coarse.edges():
        # Ordenar el par para tratar (a,b) y (b,a) como la misma tarea de fusión
        pair = tuple(sorted((a_node, b_node)))
        if pair in scored_pairs_set:
            continue
        scored_pairs_set.add(pair)

        a, b = pair[0], pair[1]
        idx_a, idx_b = node_to_idx[a], node_to_idx[b]
        u_a, u_b = u[idx_a], u[idx_b]
        v_a, v_b = v[idx_a], v[idx_b]

        beta1 = G_coarse.get_edge_data(a, b, default={'weight': 0})['weight']
        beta2 = G_coarse.get_edge_data(b, a, default={'weight': 0})['weight']

        uT_co = ((1 + beta2) / 2) * (lambda_val * u_a - beta1 * u_b) + ((1 + beta1) / 2) * (lambda_val * u_b - beta2 * u_a)
        numerator = -lambda_val * (u_a * v_a + u_b * v_b) + v_a * uT_co + beta2 * u_a * v_b + beta1 * u_b * v_a
        denominator = v_dot_u - (u_a * v_a + u_b * v_b)

        if denominator != 0:
            score = numerator / denominator
            scores.append((score, pair))

    sorted_tasks = sorted(scores, key=lambda x: x[0])

    # --- Lógica de Fusión Jerárquica ---
    node_map = {node: node for node in G.nodes()}

    def get_original_nodes(supernode):
        """Función auxiliar para obtener los nodos originales de un supernodo."""
        if not isinstance(supernode, tuple):
            return {supernode}
        return get_original_nodes(supernode[0]) | get_original_nodes(supernode[1])

    num_contractions_to_perform = int(alpha * G.number_of_nodes())
    contractions_done = 0

    print("Tareas de fusión ordenadas por score (de mejor a peor):")
    for score, (a,b) in sorted_tasks:
        print(f"  Score({a},{b}) = {score:.4f}")
    print("-" * 25)

    for score, original_pair in sorted_tasks:
        if contractions_done >= num_contractions_to_perform:
            break

        a_orig, b_orig = original_pair

        # Encuentra los supernodos a los que pertenecen a y b actualmente
        super_a = node_map[a_orig]
        super_b = node_map[b_orig]

        # Si ya pertenecen al mismo supernodo, no hay nada que hacer.
        if super_a == super_b:
            continue

        # Realiza la contracción en los supernodos.
        print(f"Fusionando {super_a} y {super_b} (basado en el score original de {original_pair})")
        G_coarse = _contract_pair(G_coarse, super_a, super_b)
        new_supernode = (super_a, super_b)

        # Actualiza el mapeo para todos los nodos originales involucrados.
        nodes_in_new_supernode = get_original_nodes(new_supernode)
        for node in nodes_in_new_supernode:
            node_map[node] = new_supernode

        lambda_track, _, _ = _calculate_eigensystem(G_coarse, list(G_coarse.nodes()))
        print(f"  --> Nuevo autovalor más grande: {lambda_track:.4f}")

        contractions_done += 1

    return G_coarse

In [5]:
# --- Ejemplo Específico para Crear un Supernodo de 3 Elementos ---
if __name__ == "__main__":
    G_test = nx.DiGraph()

    # Conexión muy fuerte entre 1 y 2 para que su score sea el mejor.
    G_test.add_edge(1, 2, weight=10)
    G_test.add_edge(2, 1, weight=0.1)

    # Conexión media entre 2 y 3 para que sea la siguiente mejor fusión.
    G_test.add_edge(2, 3, weight=5)
    G_test.add_edge(3, 2, weight=0.1)

    # Conexión débil para que no se elija.
    G_test.add_edge(3, 4, weight=0.1)
    G_test.add_edge(4, 3, weight=0.1)

    print("--- GRAFO ORIGINAL DE PRUEBA ---")
    print("Nodos:", G_test.nodes())
    print("Aristas:", G_test.edges(data=True))
    print("-" * 25)

    # Queremos 2 contracciones para fusionar 3 nodos.
    # 1. {1},{2} -> {1,2}
    # 2. {1,2},{3} -> {{1,2},3}
    G_final = coarsenet(G_test, alpha=0.5)

    print("\n--- GRAFO FINAL ---")
    print("Nodos resultantes:", G_final.nodes())
    print("Aristas resultantes:", G_final.edges(data=True))

--- GRAFO ORIGINAL DE PRUEBA ---
Nodos: [1, 2, 3, 4]
Aristas: [(1, 2, {'weight': 10}), (2, 1, {'weight': 0.1}), (2, 3, {'weight': 5}), (3, 2, {'weight': 0.1}), (3, 4, {'weight': 0.1}), (4, 3, {'weight': 0.1})]
-------------------------
Autovalor inicial (Lambda): 1.2261
Tareas de fusión ordenadas por score (de mejor a peor):
  Score(1,2) = -0.6672
  Score(2,3) = -0.5510
  Score(3,4) = -0.1109
-------------------------
Fusionando 1 y 2 (basado en el score original de (1, 2))
  --> Nuevo autovalor más grande: 1.2339
Fusionando (1, 2) y 3 (basado en el score original de (2, 3))
  --> Nuevo autovalor más grande: 0.2740

--- GRAFO FINAL ---
Nodos resultantes: [4, ((1, 2), 3)]
Aristas resultantes: [(4, ((1, 2), 3), {'weight': 0.0527}), (((1, 2), 3), 4, {'weight': 1.425})]
